In [8]:
import requests
import urllib3
import sys
import time
from datetime import datetime

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# ===== Step 1: Get Token =====
TOKEN_URL = 'https://qa.onprem.icedq.com/auth/realms/icedq.qa/protocol/openid-connect/token'
token_data = {
    'grant_type': 'password',
    'client_id': 'icedq.admin-ui',
    'client_secret': 'b2e25665-7fce-49fa-9c63-f7c872dabcf2',
    'username': 'tapshyu.g',
    'password': 'Admin@12345'
}
token_response = requests.post(TOKEN_URL, data=token_data, verify=False)
access_token = token_response.json().get("access_token")
if not access_token:
    print("❌ Failed to get token.")
    sys.exit(1)

# ===== headers for both APIs =====
Generate_entities_HEADERS = {
    'accept': 'application/json, text/plain, */*',
    'content-type': 'application/json',
    'org-id': 'org-icedq.qa',
    'workspace-id': 'wksc-36655681-7ca9-58e1-b119-71ddb41cad1e',
    'authorization': f'Bearer {access_token}',
    'account-id': 'acct-6f3fdf4c-9c27-5420-a6d5-476d2549dff6',
    'template-id':'rgen-053becce-d7c4-562a-907f-b7f87aab6297'
}
publish_entities_HEADERS = {
    'accept': 'application/json, text/plain, */*',
    'content-type': 'application/json',
    'org-id': 'org-icedq.qa',
    'authorization': f'Bearer {access_token}'
}

# ===== API URLs =====
CREATE_ENTITIES_URL = "https://qa.onprem.icedq.com/api/v1/bulk/entities/generate"
PUBLISH_ENTITIES_URL = "https://qa.onprem.icedq.com/api/v1/bulk/entities/publish"

# ===== Ask user for number of runs =====
try:
    num_runs = int(input("Enter how many times to run the API: "))
except ValueError:
    print("Invalid input! Please enter a number.")
    sys.exit(1)

# ===== Loop through API runs =====
for i in range(num_runs):
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

    # Step 2: Payload for "generate" API
    entites_payload = {
        "name": "Validation - Python",
        "description": "This template creates validation rule with pre-defined expressions based on datatype.",
        "templateType": "dynamic-sql",
        "entityType": "Validation",
        "ruleMetadata": {
            "code": "CODE00",
            "name": f"VALIDATION_${{database}}.brg.${{tables}}_{timestamp}",
            "description": "This template creates validation rule with pre-defined expressions based on datatype.",
            "purpose": "This template creates validation rule with pre-defined expressions based on datatype.",
            "criticality": "Warning",
            "customFields": []
        },
        "sql": {"source": "select * from ${tables}"},
        "columns": [
            {
                "sourceDataType": "",
                "sourceColumnFunctions": [],
                "checks": [
                    {
                        "name": "ch_${column}",
                        "description": "",
                        "type": "Custom",
                        "expression": {
                            "value": "S.[${column}] != null",
                            "emptyStringAsNull": "true",
                            "whiteSpaceAsNull": "true",
                            "trimmingEnable": "true",
                            "ignoreDataType": "true",
                            "ignoreSpecialCharacter": "false",
                            "caseInsensitive": "true",
                            "sourceColumn": "S.[${column}]"
                        },
                        "customFields": []
                    }
                ],
                "ignoreColumns": [],
                "columnNamePattern": ""
            }
        ],
        "connectionsInfo": {
            "source": {
                "connectionType": "database",
                "connectionId": "conn-f829041d-59e9-5fc4-bff5-6e2956a80b62",
                "schema": "REGRESSION_DATABASE",
                "useCached": "true"
            },
            "tablesInfo": {"sourceQualifier": "REGRESSION_DATABASE", "tables": ["STAFF"]}
        },
        "subscriptions": {
            "email": {"onSuccess": {"users": [], "groups": []},
                      "onFailure": {"users": [], "groups": []},
                      "onError": {"users": [], "groups": []}},
            "slack": {"onSuccess": {"channels": []},
                      "onFailure": {"channels": []},
                      "onError": {"channels": []}},
            "teams": {"onSuccess": {"channels": []},
                      "onFailure": {"channels": []},
                      "onError": {"channels": []}}
        },
        "ruleProperties": {
            "parameterId": "",
            "parameterFileName": "",
            "stopAtFailAndErrorCount": 22000,
            "stopAtRowCount": 1000,
            "stopAtTime": 40,
            "degreeOfParallelism": 1,
            "storeOnlyException": "true",
            "storeOnlyExportedFile": "true",
            "exportFileformat": "XLSX",
            "exportFileRowLimit": "20000",
            "createExportDownloadLink": "true",
            "appendOrderBy": "true",
            "overrideExitCode": {"overrideResult": "false", "overrideExpression": ""}
        },
        "targetContainer": {
            "workspaceId": "wksc-36655681-7ca9-58e1-b119-71ddb41cad1e",
            "folderId": "fldr-b6ddb64c-97f2-52a9-8911-227bdb02cac7",
            "folderPath": "/Ticket_Verifications/DAt-13336"
        },
        "dummyFilter": ""
    }

    print(f"\n➡️ [{i+1}/{num_runs}] Creating entity with timestamp {timestamp} ...")
    create_response = requests.post(CREATE_ENTITIES_URL, headers=Generate_entities_HEADERS, json=entites_payload, verify=False)

    if create_response.status_code != 200:
        print(f"❌ Create API failed ({create_response.status_code}): {create_response.text}")
        continue

    data = create_response.json()
    payload_data = data.get("payload", {})
    success_data = payload_data.get("success", [])

    if not success_data:
        print("⚠️ No success data found in response.")
        continue

    entity_id = success_data[0].get("entityId")
    request_id = success_data[0].get("requestId")

    print(f"✅ Created Entity: entityId={entity_id}, requestId={request_id}")

    # Step 3: Publish API call
    publish_payload = {
        "entities": [
            {
                "entityId": entity_id,
                "requestId": request_id
            }
        ]
    }

    print(f"➡️ Publishing entity {entity_id} ...")
    publish_response = requests.post(PUBLISH_ENTITIES_URL, headers=publish_entities_HEADERS, json=publish_payload, verify=False)

    if publish_response.status_code == 200:
        print(f"✅ Published Successfully: {publish_response.json()}")
    else:
        print(f"❌ Publish API failed ({publish_response.status_code}): {publish_response.text}")

    # Optional delay to avoid flooding server
    time.sleep(1)



➡️ [1/2] Creating entity with timestamp 20251029_104048 ...
✅ Created Entity: entityId=enty-bb74f8f6-3c60-5c60-9cce-dc0cda9c226d, requestId=rqst-eda81f80-4d6d-5782-b7c5-39e35a6dbc44
➡️ Publishing entity enty-bb74f8f6-3c60-5c60-9cce-dc0cda9c226d ...
✅ Published Successfully: [{'name': 'VALIDATION_brg_STAFF_20251029_104048', 'status': 'Published', 'message': 'Rule published successfully', 'requestId': 'rqst-eda81f80-4d6d-5782-b7c5-39e35a6dbc44', 'objectId': 'rule-f12a8442-a2e5-5c18-85d6-df9ac563f09c', 'entityId': 'enty-bb74f8f6-3c60-5c60-9cce-dc0cda9c226d', 'publishDttm': '2025-10-29T05:11:01.182847378Z'}]

➡️ [2/2] Creating entity with timestamp 20251029_104104 ...
✅ Created Entity: entityId=enty-a50da989-6821-5003-a35d-da076a296c5f, requestId=rqst-2964f28b-8dd9-56cd-96bc-1dc2bbe35703
➡️ Publishing entity enty-a50da989-6821-5003-a35d-da076a296c5f ...
✅ Published Successfully: [{'name': 'VALIDATION_brg_STAFF_20251029_104104', 'status': 'Published', 'message': 'Rule published successfull